In [86]:
import torch
import torchvision
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.transforms as transforms
from torchvision.utils import save_image as save_image
import matplotlib.pyplot as plt
import os


In [32]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))]) #1 channel transform

trainset = torchvision.datasets.MNIST(root='./data/MNIST', train=True, #MNIST Dataset
                                    download=True, transform=transform) 
train_loader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                         shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data/MNIST', train=False,
                                    download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)


In [63]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)
    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))
    
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [64]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False)
    KLD = -0.5*torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [87]:
def create_dir(directory): #for result dir 
    if not os.path.exists(directory):
        os.makedirs(directory)

In [88]:
model = VAE()
device = torch.device("cpu")
optimizer = optim.Adam(model.parameters(), lr=0.0001)


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item() / len(data)))
            print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [89]:
create_dir('./results_VAE')

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0),8)
                comparison = torch.cat([data[:n], recon_batch.view(4, 1, 28, 28)[:n]])
                save_image(comparison.cpu(), './results_VAE/reconstruction_' + str(epoch) + '.png', nrow=n)
                
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
            

In [91]:
for epoch in range(1, 10 + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1,28, 28), './results_VAE/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: -20857.392578
====> Epoch: 1 Average loss: -1.3905
Train Epoch: 1 [40000/60000 (67%)]	Loss: -21343.193359
====> Epoch: 1 Average loss: -13653.7506
====> Test set loss: -20505.4692
Train Epoch: 2 [0/60000 (0%)]	Loss: -19351.296875
====> Epoch: 2 Average loss: -1.2901
Train Epoch: 2 [40000/60000 (67%)]	Loss: -21428.777344
====> Epoch: 2 Average loss: -13600.7105
====> Test set loss: -20510.9699
Train Epoch: 3 [0/60000 (0%)]	Loss: -20506.050781
====> Epoch: 3 Average loss: -1.3671
Train Epoch: 3 [40000/60000 (67%)]	Loss: -19416.246094
====> Epoch: 3 Average loss: -13567.5318
====> Test set loss: -20350.9978
Train Epoch: 4 [0/60000 (0%)]	Loss: -20356.216797
====> Epoch: 4 Average loss: -1.3571
Train Epoch: 4 [40000/60000 (67%)]	Loss: -20595.064453
====> Epoch: 4 Average loss: -13530.2848
====> Test set loss: -20302.6645
Train Epoch: 5 [0/60000 (0%)]	Loss: -20530.935547
====> Epoch: 5 Average loss: -1.3687
Train Epoch: 5 [40000/60000 (67%)]	Loss: -18972.8

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()